推荐系统 - 一种信息过滤系统 利用一定规则给物品排序 解决如下问题： 
- 是否构成信息过载      过载
- 用户意图是否明确      不明确

搜索引擎： 流量分布呈现出马太效应 目标是快速满足用户需求 评估指标简单 个性化弱   用户主动搜索 \
推荐系统： 流量分布呈现出长尾效应 目标是长时间持续服务   评估指标复杂 个性化较强 用户被动接受


推荐系统工作原理
- 社会化推荐
- 基于内容推荐
- 基于流行程度推荐
- 基于协同过滤的推荐 （人与群分）

Web项目和推荐系统的区别：
Web项目： 提供稳定、支持高并发、可预测性的信息流通道服务 \
推荐系统： 追求增长指标（GMV、成交金额）结果是概率的结果

推荐系统设计
- UI
- 数据（Lambda架构）
- 业务知识
- 算法

推荐系统架构 \
Lambda架构采集数据 --> 推荐算法计算推荐结果 --> 用户服务 --> 用户行为反馈 --> Lambda架构数据

大数据Lambda架构：（Batch Layer、Speed Layer、Serving Layer）使用离线和实时数据混合提供服务 \

- 分层架构
    - 批处理层 （离线数据） 数据量大、处理速度慢
        - 分布式存储 Hadoop distributive file system
        - 分布式计算 Hadoop MapReduce & spark
        - 存储
        - 日志收集 Flume 数据搬运
    - 实时处理层  响应速度快 （ms级别响应） 数据量小
        - 流式计算
        - 实时数据分析 spark streaming / storm / flink
        - 消息队列 kafka

离线计算的框架
- hadoop 作为批处理层的核心计算框架，处理全量历史数据
    - 使用HDFS存储所有原始数据
    - 通过MapReduce进行大规模数据的批量处理
    - 处理后的结果存储在服务层供查询
- spark core 替代或补充Hadoop MapReduce，提供更快的批处理能力
- spark sql 在批处理层提供SQL接口处理结构化数据
- hive 数据仓库工具，用于批处理层的数据管理和查询，将处理结果输出到服务层存储

实时计算框架
- spark streaming 处理实时数据流，提供近实时的计算结果
- storm
- flink

消息中间件
- flume 日志采集系统 负责数据收集和传输
    - 从各种数据源收集日志和事件数据
    - 将数据传输到Kafka或其他存储系统
    - 在Lambda架构中通常作为数据入口

- kafka消息队列    
    - 分布式消息队列，作为数据缓冲和分发中心
    - 接收来自Flume等数据源的数据
    - 为批处理层和速度层提供统一的数据源

- 存储相关
- hbase （nosql数据库）常用于服务层存储
- hive  （sql操作hdfs数据）
    - 存储结构化数据，提供SQL接口
    - 管理数据仓库中的历史数据

Lambda架构中各组件的协作流程

- 数据摄入：

    Flume收集日志/事件数据 → 发送到Kafka

    其他数据源也可能直接写入Kafka或HDFS

- 批处理层：

    定期(如每天)将Kafka或原始数据加载到HDFS

    使用Hadoop/Spark/Hive处理全量数据

    计算结果写入HBase等服务层存储

- 速度层：

    Spark Streaming/Storm/Flink实时消费Kafka数据

    处理最新数据，生成增量结果

    将实时视图写入高速存储(如Redis)或HBase

- 服务层：

    合并批处理视图和实时视图

    响应查询时，将批处理结果与实时更新合并

    通过HBase或专用服务数据库提供查询接口

- 数据流：
    原始数据 → Kafka → 同时供给:

    批处理层(Spark/Hadoop): 高延迟高准确度处理

    速度层(Spark Streaming/Flink/Storm): 低延迟近似处理
    结果合并 → 服务层 → 用户查询